<a href="https://colab.research.google.com/github/sandrorjunior/public/blob/main/tcc_pucrs_r00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Importar bibliotecas que serão utilizadas

In [ ]:
import requests
import zipfile
import io
import pandas as pd
from google.colab import files
import plotly.graph_objects as go
import datetime


2. Baixar do servidor online dados atualizados

In [ ]:
#configuracoes para o csv extraido
pd.set_option('display.float_format', '{:.2f}'.format)

# URL do arquivo zip
url = "https://repositorio.dados.gov.br/seges/detru/siconv.zip"

# Faz o download do arquivo zip
response = requests.get(url)
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
# Lista de arquivos no zip
file_names = zip_file.namelist()

# Diretório de destino para salvar os arquivos
destination_dir = "/content/zip_files/"

# Extrai cada arquivo do zip e salva no diretório de destino
for file_name in file_names:
    zip_file.extract(file_name, destination_dir)

**3**. Transformar csvs que serão utilizados em dataframes panda

In [ ]:
#funcao para extrair dataframes
def extrair_df(file_name):
    # Diretório de destino
    destination_dir = "/content/zip_files/"

    # Caminho completo do arquivo CSV
    csv_file_path = destination_dir + file_name

    # Carrega o arquivo CSV em um DataFrame
    df = pd.read_csv(csv_file_path, sep=';')

    # Retorna o DataFrame
    return df

In [ ]:
#extrairdataframes
convenio = extrair_df("siconv_convenio.csv")
proposta = extrair_df("siconv_proposta.csv")
inst_cont_proposta_aio_modulo_empresas = extrair_df("siconv_inst_cont_proposta_aio_modulo_empresas.csv")
vrpl_proposta_licitacao_modulo_empresas = extrair_df("siconv_vrpl_proposta_licitacao_modulo_empresas.csv")
historico_situacao = extrair_df("siconv_historico_situacao.csv")

historico_situacao['DIA_HISTORICO_SIT'] = pd.to_datetime(historico_situacao['DIA_HISTORICO_SIT'], dayfirst=True, errors='coerce')

In [ ]:
#criar dataframes com dia de pcf a partir do df historico_situacao
data_pcf = historico_situacao
data_pcf = data_pcf.loc[data_pcf['HISTORICO_SIT'] == "PRESTACAO_CONTAS_CONCLUIDA"]
data_pcf = data_pcf.groupby('ID_PROPOSTA')['DIA_HISTORICO_SIT'].max().reset_index()
data_pcf = data_pcf.rename(columns={'DIA_HISTORICO_SIT': 'DIA_PCF'})

# Agora a coluna 'DIA_HISTORICO_SIT' será renomeada para 'DIA_PCF' em df1.

#criar dataframe com ultimo_dia_historico
# Primeiro, você precisa agrupar o DataFrame historico por 'ID_PROPOSTA' e encontrar a data máxima para cada grupo.
ultimo_dia_historico = historico_situacao
ultimo_dia_historico = ultimo_dia_historico.dropna(subset=['DIA_HISTORICO_SIT'])
ultimo_dia_historico = ultimo_dia_historico.groupby('ID_PROPOSTA')['DIA_HISTORICO_SIT'].max().reset_index()
ultimo_dia_historico = ultimo_dia_historico.rename(columns={'DIA_HISTORICO_SIT': 'ULTIMO_DIA_HISTORICO_SIT'})




4. Criar dataframe final pro estudo de prazos

In [ ]:
#criar df1 com as colunas relevantes para o estudo de prazo dos contratos

# Selecione as colunas que você deseja em df1
cols_convenio = ['NR_CONVENIO', 'ID_PROPOSTA', 'DIA_ASSIN_CONV', 'SIT_CONVENIO', 'INSTRUMENTO_ATIVO', 'VL_GLOBAL_CONV', 'VL_REPASSE_CONV', 'VL_CONTRAPARTIDA_CONV', 'DATA_SUSPENSIVA', 'DATA_RETIRADA_SUSPENSIVA', 'DIA_FIM_VIGENC_CONV']
cols_proposta = ['ID_PROPOSTA', 'UF_PROPONENTE', 'MUNIC_PROPONENTE', 'COD_MUNIC_IBGE', 'COD_ORGAO_SUP', 'DIA_PROPOSTA', 'MODALIDADE', 'OBJETO_PROPOSTA']
cols_inst_cont_proposta = ['ID_PROPOSTA', 'DATA_EMISSAO_AIO_INSTRUMENTO_CONTRATUAL']
cols_vrpl_proposta = ['ID_PROPOSTA', 'DATA_ACEITE_LICITACAO_VRPL']
cols_data_pcf = ['ID_PROPOSTA', 'DIA_PCF']
cols_ultimo_dia_historico = ['ID_PROPOSTA', 'ULTIMO_DIA_HISTORICO_SIT']

# Combine os DataFrames
df1 = convenio[cols_convenio]
df1 = pd.merge(df1, proposta[cols_proposta], on='ID_PROPOSTA', how='left')
df1 = pd.merge(df1, inst_cont_proposta_aio_modulo_empresas[cols_inst_cont_proposta], on='ID_PROPOSTA', how='left')
df1 = pd.merge(df1, vrpl_proposta_licitacao_modulo_empresas[cols_vrpl_proposta], on='ID_PROPOSTA', how='left')
df1 = pd.merge(df1, data_pcf[cols_data_pcf], on='ID_PROPOSTA', how='left')
df1 = pd.merge(df1, ultimo_dia_historico[cols_ultimo_dia_historico], on='ID_PROPOSTA', how='left')

#apenas modalidade contrato de repasse
df1 = df1[df1['MODALIDADE'] == 'CONTRATO DE REPASSE']

#converter tipos de variáveis
colunas_datetime = ['DIA_ASSIN_CONV', 'DATA_SUSPENSIVA', 'DATA_RETIRADA_SUSPENSIVA', 'DIA_FIM_VIGENC_CONV', 'DIA_PROPOSTA', 'DATA_EMISSAO_AIO_INSTRUMENTO_CONTRATUAL', 'DATA_ACEITE_LICITACAO_VRPL']
df1[colunas_datetime] = df1[colunas_datetime].apply(pd.to_datetime, errors='coerce')
colunas_float = ['VL_GLOBAL_CONV', 'VL_REPASSE_CONV', 'VL_CONTRAPARTIDA_CONV']
df1[colunas_float] = df1[colunas_float].apply(pd.to_numeric, errors='coerce')

#coluna data de encerramento
df1['INSTRUMENTO_ATIVO'] = df1['INSTRUMENTO_ATIVO'].str.strip()
def calcular_data_encerramento(row):
    if row['INSTRUMENTO_ATIVO'] == 'NÃO':
        return row['ULTIMO_DIA_HISTORICO_SIT']
    else:
        return pd.NaT  # Isso representa um valor nulo em Pandas
# Aplicando a função à coluna 'DATA_ENCERRAMENTO'
df1['DATA_ENCERRAMENTO'] = df1.apply(calcular_data_encerramento, axis=1)

#criar colunas de status para fazer o diagrama
df1['STATUS_ASSINATURA'] = df1['DIA_ASSIN_CONV'].apply(lambda x: 'Contrato assinado' if pd.notna(x) else 'Contrato não assinado')
# Exclude rows where 'STATUS_ASSINATURA' is equal to 'Contrato não assinado'
df1 = df1[df1['STATUS_ASSINATURA'] != 'Contrato não assinado']
def calcular_status(row):
    if row['SIT_CONVENIO'] in ['Cancelado', 'Convênio Anulado', 'Convênio Rescindido', 'Prestação de Contas Rejeitada']:
        return 'Finalizado sem sucesso'
    elif pd.notna(row['DATA_ENCERRAMENTO']):
        return 'Finalizado com sucesso'
    elif row['STATUS_ASSINATURA'] == 'Contrato não assinado':
        return 'Contrato a assinar'
    else:
        return 'Contrato em andamento'
df1['STATUS_FINALIZACAO'] = df1.apply(calcular_status, axis=1)
hoje = datetime.datetime.now()
def calcular_status_suspensiva(row):
    if row['STATUS_FINALIZACAO'] == 'Contrato a assinar':
        return 'Contrato a assinar '
    elif pd.notna(row['DATA_RETIRADA_SUSPENSIVA']):
        return 'Suspensiva retirada'
    elif pd.notna(row['DATA_SUSPENSIVA']) and row['DATA_SUSPENSIVA'] < hoje:
        return 'Prazo de suspensiva perdido'
    elif row['DATA_SUSPENSIVA'] >= hoje and row['STATUS_FINALIZACAO'] == 'Contrato em andamento':
        return 'Aguarda retirada de suspensiva'
    elif row['STATUS_FINALIZACAO'] == 'Finalizado sem sucesso':
        return 'Finalizado sem sucesso '
    else:
        return 'Sem info de Suspensiva'
df1['STATUS_SUSPENSIVA'] = df1.apply(calcular_status_suspensiva, axis=1)
def calcular_status_vrpl(row):
    if row['STATUS_FINALIZACAO'] == 'Contrato a assinar':
        return 'Contrato a assinar  '
    elif pd.to_datetime(row['DATA_RETIRADA_SUSPENSIVA']) > pd.to_datetime('2019-01-01') and pd.isna(row['DATA_ACEITE_LICITACAO_VRPL']) and row['STATUS_FINALIZACAO'] == 'Contrato em andamento':
       return 'Aguarda VRPL'
    elif pd.notna(row['DATA_ACEITE_LICITACAO_VRPL']):
        return 'VRPL emitida'
    elif row['STATUS_SUSPENSIVA'] == 'Aguarda retirada de suspensiva':
        return 'Aguarda VRPL'
    elif row['STATUS_FINALIZACAO'] == 'Finalizado sem sucesso':
        return 'Finalizado sem sucesso  '
    else:
        return 'Sem info de VRPL'
df1['STATUS_VRPL'] = df1.apply(calcular_status_vrpl, axis=1)
def calcular_status_aio(row):
    if row['STATUS_FINALIZACAO'] == 'Contrato a assinar':
        return 'Contrato a assinar   '
    elif row['STATUS_VRPL'] == 'VRPL emitida' and pd.isna(row['DATA_EMISSAO_AIO_INSTRUMENTO_CONTRATUAL']):
        return 'Aguarda AIO'
    elif row['STATUS_VRPL'] == 'Aguarda VRPL':
        return 'Aguarda AIO'
    elif pd.notna(row['DATA_EMISSAO_AIO_INSTRUMENTO_CONTRATUAL']) and pd.notna(row['DATA_ACEITE_LICITACAO_VRPL']):
        return 'AIO emitida'
    elif row['STATUS_FINALIZACAO'] == 'Finalizado sem sucesso':
        return 'Finalizado sem sucesso   '
    else:
        return 'Sem info de AIO'
df1['STATUS_AIO'] = df1.apply(calcular_status_aio, axis=1)
#limpar com contratos a assinar


5. Criar visualização via diagrama de sankey

In [ ]:
#https://github.com/bhattbhavesh91/sankey-diagram-plotly-tutorial/blob/main/sankey-diagram-notebook.ipynb
#primeiro por quantidade de operações
#criar dataframes de origem e destino
df2 = df1.groupby(['STATUS_ASSINATURA', 'STATUS_SUSPENSIVA'])['ID_PROPOSTA'].count().reset_index()
df2.columns = ['source', 'target', 'value']
df3 = df1.groupby(['STATUS_SUSPENSIVA', 'STATUS_VRPL'])['ID_PROPOSTA'].count().reset_index()
df3.columns = ['source', 'target', 'value']
df4 = df1.groupby(['STATUS_VRPL', 'STATUS_AIO'])['ID_PROPOSTA'].count().reset_index()
df4.columns = ['source', 'target', 'value']
df5 = df1.groupby(['STATUS_AIO', 'STATUS_FINALIZACAO'])['ID_PROPOSTA'].count().reset_index()
df5.columns = ['source', 'target', 'value']

links = pd.concat([df2, df3, df4, df5], axis=0)

#criando lista de valores unicos de situações
unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel('K')))

#criando dicionário atribuindo um número para cada valor único de situação
mapping_dict = {k: v for v, k in enumerate(unique_source_target)}

#transformando a links em valores atribuidos no dicionario
links['source'] = links['source'].map(mapping_dict)
links['target'] = links['target'].map(mapping_dict)

#criando link em formato lista para usar no fig =
links_dict = links.to_dict(orient='list')

#criando figura
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = unique_source_target,
      color = "blue"
    ),
    link = dict(
      source = links_dict["source"],
      target = links_dict["target"],
      value = links_dict["value"]

  ))])



# Customize the layout of the Sankey diagram, including the title
fig.update_layout(
    title_text="Diagrama de Sankey - Contratos de Repassse no portal TrasfereGOV - Quantidade de operações",
    font_size=10,
)

fig.show()



In [ ]:
#segundo por soma do valor de repasse de operações


#criar dataframes de origem e destino
df2 = df1.groupby(['STATUS_ASSINATURA', 'STATUS_SUSPENSIVA'])['VL_REPASSE_CONV'].sum().reset_index()
df2.columns = ['source', 'target', 'value']
df3 = df1.groupby(['STATUS_SUSPENSIVA', 'STATUS_VRPL'])['VL_REPASSE_CONV'].sum().reset_index()
df3.columns = ['source', 'target', 'value']
df4 = df1.groupby(['STATUS_VRPL', 'STATUS_AIO'])['VL_REPASSE_CONV'].sum().reset_index()
df4.columns = ['source', 'target', 'value']
df5 = df1.groupby(['STATUS_AIO', 'STATUS_FINALIZACAO'])['VL_REPASSE_CONV'].sum().reset_index()
df5.columns = ['source', 'target', 'value']

links = pd.concat([df2, df3, df4, df5], axis=0)

#criando lista de valores unicos de situações
unique_source_target = list(pd.unique(links[['source', 'target']].values.ravel('K')))

#criando dicionário atribuindo um número para cada valor único de situação
mapping_dict = {k: v for v, k in enumerate(unique_source_target)}

#transformando a links em valores atribuidos no dicionario
links['source'] = links['source'].map(mapping_dict)
links['target'] = links['target'].map(mapping_dict)

#criando link em formato lista para usar no fig =
links_dict = links.to_dict(orient='list')

#criando figura
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = unique_source_target,
      color = "blue"
    ),
    link = dict(
      source = links_dict["source"],
      target = links_dict["target"],
      value = links_dict["value"]

  ))])



# Customize the layout of the Sankey diagram, including the title
fig.update_layout(
    title_text="Diagrama de Sankey - Contratos de Repassse no portal TrasfereGOV - Soma de valor de repasse",
    font_size=10,
)

fig.show()